In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction
from chromalab.animation import *

objc[85477]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2b2386e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2ce71eea0). One of the two will be used. Which one is undefined.
objc[85477]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2b2386db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2ce71eec8). One of the two will be used. Which one is undefined.
objc[85477]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2b2386e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths5, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
viz = PSWrapper(trichromat, maxbasis_tri, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.CONE, verbose=True)

# tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
# maxbasis = MaxBasis(tetrachromat, verbose=True)
# viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True) 

100%|██████████| 58/58 [00:00<00:00, 644.39it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 3660/3660 [00:08<00:00, 417.70it/s]


In [7]:
# general settings - RUN THIS CELL TO RESET AN ANIMATION RUN
viz.ps.set_ground_plane_mode("none")
dirname = "./outputs/trichromat_obs"
os.makedirs(dirname, exist_ok=True)
offset = 0 # start at 0 frames indexing

In [8]:
# render all objects
viz.renderObjectsPS(mesh_alpha=0.8, lattice_alpha=1)

# transform to correct dir
mat = np.eye(4)
mat[:3, :3] = viz.HMatrix[::-1]
viz.ps.get_surface_mesh("mesh").set_transform(mat)
viz.ps.get_surface_mesh("lattice").set_transform(mat)

In [9]:
list_names = ["mesh", "lattice"]
set_enabled = SetEnabledFn(viz, list_names)
long_frames = 90
medium_frames = 60
short_frames = 30

In [10]:
r, theta, phi = 4, 90, 0
origin = np.array([0, 0, np.sqrt(3)/2])
# xyz = PSWrapper.polarToCartesian(r, theta, phi)
# viz.ps.look_at(xyz)

In [11]:
# 1. Point Cloud and then Render Mesh
points = trichromat.get_optimal_colors()[0]
points = (viz.HMatrix[::-1]@points.T).T
rgbs = trichromat.get_optimal_rgbs()

100%|██████████| 3660/3660 [00:06<00:00, 529.67it/s]


In [12]:
LMSinHering = viz.HMatrix[::-1]@np.eye(3)
viz._getCoordBasis('LMS',  LMSinHering.T, coordAlpha=1)
name_points = viz.renderPointCloud(points, rgbs, radius=0.01)[0]

In [13]:
superLongFrames = 2*long_frames
rotZLong = RotateAroundZ(viz, superLongFrames, r, theta, [0,360], lookAt=origin)

In [14]:

def opacityWhileRotate(i):
    set_enabled([True, False])
    viz.ps.get_surface_mesh("mesh").set_transparency(0.8 * easeFunction(i/superLongFrames))
    viz.ps.get_point_cloud(name_points).set_transparency(1- easeFunction(i/superLongFrames))
    if i == superLongFrames - 5:
        viz.ps.get_point_cloud(name_points).set_enabled(False)
    rotZLong(i)
    return i

offset = viz.renderScreenshots(opacityWhileRotate, dirname, superLongFrames, offset)

In [15]:
LMStoRGB, last_transform = CoordBasisChange(viz, superLongFrames)
rotZFirst = RotateAroundZ(viz, medium_frames, r, theta, [0,65], lookAt=origin)
rotZAfter= RotateAroundZ(viz, superLongFrames, r, theta, [65,360], lookAt=origin)

viz.ps.remove_surface_mesh("LMS")
offset = viz.renderScreenshots(rotZFirst, dirname, medium_frames, offset)
offset = viz.renderScreenshots(LMStoRGB, dirname, superLongFrames, offset)
offset = viz.renderScreenshots(rotZAfter, dirname, superLongFrames, offset)

In [16]:
lum_name = viz.renderLuminanceAxis()

fadeLumAxis = FadeIn(viz, lum_name[0], medium_frames, [0.2, 1])
fadeOutLMSCoords = FadeOut(viz, "LMS_coords", medium_frames, [0.2, 1], removeBefore=5)
rotZMedium= RotateAroundZ(viz, medium_frames, r, theta, [0,90], lookAt=origin)
fadeLumRotateInMedium = concatFns([fadeLumAxis, rotZMedium, fadeOutLMSCoords])
offset = viz.renderScreenshots(fadeLumRotateInMedium, dirname, medium_frames, offset)

In [17]:
rotZMedium3= RotateAroundZ(viz, medium_frames * 3, r, theta, [90,360], lookAt=origin)
offset = viz.renderScreenshots(rotZMedium3, dirname, medium_frames * 3, offset)

In [18]:
# squash along L Axis
def Squash(i):
    viz.renderFlattenExistingMesh(mesh_alpha=0.8, height=1 - (i/long_frames), basis=viz.ps.get_surface_mesh("mesh").get_transform()[:3, :3])
fade_coords = FadeOut(viz, "RGB_coords", long_frames, opacity_range=[0.2, 1], removeBefore=5)

set_enabled([False, False])
squash_fade = concatFns([fade_coords, Squash])
offset = viz.renderScreenshots(squash_fade, dirname, long_frames, offset)

In [19]:
rotTheta = RotateTheta(viz, medium_frames, r, [90, 0], phi, lookAt=origin)
fade_Lum = FadeOut(viz, lum_name[0], medium_frames)
fadeInMesh = FadeIn(viz, "flatmesh", medium_frames, [0.8, 1])

fadeandRot = concatFns([fade_Lum, rotTheta, fadeInMesh])
offset = viz.renderScreenshots(fadeandRot, dirname, medium_frames, offset)

In [20]:
exportAndPlay(dirname)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 

: 